In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev_test.tsv
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install torch
!pip install scikit-learn
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from sklearn.model_selection import StratifiedKFold


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
train_file = 'blp25_hatespeech_subtask_1B_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1B_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1B_dev_test.tsv'
import os
os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./results/",  # Changed to avoid conflict
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no",
    report_to=None
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")
model_name = 'csebuetnlp/banglabert'
set_seed(training_args.seed)
l2id = {"None": 0, "Society": 1, "Organization": 2, "Community": 3, "Individual": 4}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)
print(len(test_df['id']))
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


data_collator = default_data_collator

train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

# Add stratified 5-fold CV
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store results for each fold
fold_results = []
fold_probs = []

# Get labels for splitting
labels = np.array(train_dataset["label"])

original_output_dir = training_args.output_dir

for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(len(train_dataset)), labels)):
    print(f"\n{'='*50}")
    print(f"FOLD {fold + 1}/{n_splits}")
    print(f"{'='*50}")
    
    # Create train and validation datasets for this fold
    fold_train = train_dataset.select(train_idx)
    fold_val = train_dataset.select(val_idx)
    
    print(f"Train size: {len(fold_train)}, Validation size: {len(fold_val)}")
    
    # Initialize model for this fold (fresh model each time)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        from_tf=bool(".ckpt" in model_name),
        config=config,
        cache_dir=None,
        revision="main",
        use_auth_token=None,
        ignore_mismatched_sizes=False,
    )
    
    # Update training arguments for this fold
    fold_dir = os.path.join(original_output_dir, f"fold_{fold}")
    os.makedirs(fold_dir, exist_ok=True)
    fold_training_args = TrainingArguments(
        learning_rate=2e-5,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        output_dir=fold_dir,
        overwrite_output_dir=True,
        remove_unused_columns=False,
        local_rank=1,
        load_best_model_at_end=True,
        save_total_limit=2,
        save_strategy="no",
        report_to=None
    )
    
    trainer = Trainer(
        model=model,
        args=fold_training_args,
        train_dataset=fold_train,
        eval_dataset=fold_val,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    # Train the model
    print(f"Training fold {fold + 1}...")
    train_result = trainer.train()
    
    # Evaluate the model
    print(f"Evaluating fold {fold + 1}...")
    eval_result = trainer.evaluate()
    
    # Store results
    fold_results.append({
        'fold': fold + 1,
        'train_loss': train_result.metrics.get('train_loss', None),
        'eval_loss': eval_result['eval_loss'],
        'eval_accuracy': eval_result['eval_accuracy']
    })
    
    # Generate predictions on test set for this fold
    print(f"Predicting with fold {fold + 1} model...")
    test_predictions = trainer.predict(predict_dataset.remove_columns("id") if "id" in predict_dataset.column_names else predict_dataset)
    probs = torch.softmax(torch.tensor(test_predictions.predictions), dim=-1).numpy()
    fold_probs.append(probs)
    
    # Clean up to save memory
    del model, trainer
    
    print(f"Fold {fold + 1} - Accuracy: {eval_result['eval_accuracy']:.4f}")
    
print(f"\n{'='*50}")
print("CROSS-VALIDATION COMPLETED")
print(f"{'='*50}")

# Analyze cross-validation results
import pandas as pd

results_df = pd.DataFrame(fold_results)
print("\nCross-Validation Results:")
print(results_df)

# Calculate average performance metrics
avg_train_loss = results_df['train_loss'].mean() if 'train_loss' in results_df else None
avg_eval_loss = results_df['eval_loss'].mean()
avg_acc = results_df['eval_accuracy'].mean()
std_acc = results_df['eval_accuracy'].std()

print(f"\nAverage Results Across {n_splits} Folds:")
if avg_train_loss is not None:
    print(f"Average Training Loss: {avg_train_loss:.4f}")
print(f"Average Validation Loss: {avg_eval_loss:.4f}")
print(f"Average Accuracy: {avg_acc:.4f} ± {std_acc:.4f}")

# Ensemble predictions
probs_folds = np.array(fold_probs)
ensemble_probs = np.mean(probs_folds, axis=0)

# Final ensemble prediction
final_preds = np.argmax(ensemble_probs, axis=1)

id2l = {v: k for k, v in l2id.items()}
logger.info("*** Predict ***")
ids = predict_dataset['id']
output_predict_file = os.path.join(training_args.output_dir, f"subtask_1A.tsv")
with open(output_predict_file, "w") as writer:
    logger.info(f"***** Predict results *****")
    writer.write("id\tlabel\tmodel\n")
    for index, item in enumerate(final_preds):
        item = id2l[item]
        writer.write(f"{ids[index]}\t{item}\t{model_name}\n")
print(f"\nPredictions saved to '{output_predict_file}'")

--2025-09-03 04:50:07--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8005567 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1B_train.tsv’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.08s   

2025-09-03 04:50:08 (95.8 MB/s) - ‘blp25_hatespeech_subtask_1B_train.tsv’ saved [8005567/8005567]

--2025-09-03 04:50:08--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.co

2025-09-03 04:51:57.130808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756875117.457371      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756875117.549474      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


2512
[0, 4, 1, 3, 2]


config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

[INFO|configuration_utils.py:698] 2025-09-03 04:52:19,250 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglabert/snapshots/9ce791f330578f50da6bc52b54205166fb5d1c8c/config.json
[INFO|configuration_utils.py:770] 2025-09-03 04:52:19,257 >> Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id":

...

Step,Training Loss
500,1.021000
1000,0.816800
1500,0.708500
2000,0.664500
2500,0.606900


[INFO|trainer.py:2676] 2025-09-03 06:37:11,458 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:4327] 2025-09-03 06:37:11,496 >> 
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-09-03 06:37:11,497 >>   Num examples = 7104
[INFO|trainer.py:4332] 2025-09-03 06:37:11,497 >>   Batch size = 32


Evaluating fold 5...


[INFO|trainer.py:4327] 2025-09-03 06:37:48,772 >> 
***** Running Prediction *****
[INFO|trainer.py:4329] 2025-09-03 06:37:48,773 >>   Num examples = 2512
[INFO|trainer.py:4332] 2025-09-03 06:37:48,773 >>   Batch size = 32


Predicting with fold 5 model...
Fold 5 - Accuracy: 0.7134

CROSS-VALIDATION COMPLETED

Cross-Validation Results:
   fold  train_loss  eval_loss  eval_accuracy
0     1    0.735863   0.811716       0.708797
1     2    0.746137   0.785012       0.722590
2     3    0.757679   0.778685       0.718046
3     4    0.747193   0.804745       0.705518
4     5    0.752274   0.801056       0.713401

Average Results Across 5 Folds:
Average Training Loss: 0.7478
Average Validation Loss: 0.7962
Average Accuracy: 0.7137 ± 0.0069

Predictions saved to './results/subtask_1A.tsv'
